# Gauge transformation of the gauge links in a LC coordinate lattice

## Step 1

We write the relevant gauge link in a $(x^+, x^-, y)$ lattice

In [2]:
import os

os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = 'su3'

import curraun.su as su
import numpy as np
import pickle

Using SU(3)
Using Numba
Using double precision


/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# We read the fields

fields = np.load('gauge_links_fields_glasma_test.npz')
ux = fields['ux']

with open('parameters_gauge_transf_glasma_test.pickle', 'rb') as f:
    param = pickle.load(f)

In [4]:
print(np.shape(ux))

(64, 16384, 9)


In [5]:
# We get the relevant parameters

N = param['N']
Dg = param['DG']

t_steps = np.shape(ux)[0]

In [6]:
# We reshape the transverse component into a N*N matrix

ux_r = ux.reshape((t_steps, N, N, Dg))

print(np.shape(ux_r))

(64, 128, 128, 9)


In [15]:
# We define a function that takes a Milne lattice functions and puts it in the LC lattice

def change_lattice(u):
    u_LC = su.GROUP_TYPE(np.zeros((t_steps//2, t_steps//2, N, Dg)))

    for i in range(t_steps//2):
        for j in range(t_steps//2):
            u_LC[i,j,:,:] = u[i+j,i-j+N//2,:,:]
    
    return u_LC

In [16]:
# We transform the Ux links to the LC coordinates

ux_LC = change_lattice(ux_r)

print(np.shape(ux_LC))

(32, 32, 128, 9)


In [17]:
# We get the plus and minus direction links

uminus = np.apply_along_axis(su.dagger, 3, ux_LC)
uplus = ux_LC

## Step 2

We gauge transform the field in the LC lattice

In [19]:
# We write the gauge transformation operator at a given (x^+, x^-) point

def gauge_operator(u, xplus): #The input has to be written in the Milne lattice (TODO: writte the input in LC lattice ?)
    buffer1 = np.array([su.unit() for j in range(N)])
    v = su.GROUP_TYPE(np.zeros((t_steps//2, N, Dg)))
    
    for i in range(-xplus, t_steps//2):
        buffer2 = ux_r[xplus + i, xplus - i + N//2, :, :]
        r = np.array([su.mul(buffer2[j, :], buffer1[j, :]) for j in range(N)])
        if i>0:
            v[i, :, :]
        buffer1 = r
        
    return v

In [21]:
# We create a matrix that contains the gauge transformation operator at every point

v_LC = su.GROUP_TYPE(np.zeros((t_steps//2, t_steps//2, N, Dg)))

for k in range(t_steps//2):
    v_LC[k,:,:,:] = gauge_operator(ux_r, k)

In [22]:
print(np.shape(v_LC))

(32, 32, 128, 9)
